Solution setup is covered in detail in the `basics_tutorial` so it will not be covered here.

If you are new to `solvation_analysis` check there first!

In [1]:
# imports
import MDAnalysis as mda
from solvation_analysis.solution import Solution

# define paths to data
data = "../solvation_analysis/tests/data/bn_fec.data"
traj = "../solvation_analysis/tests/data/bn_fec_short_unwrap.dcd"

# instantiate Universe
u = mda.Universe(data, traj)

# define solute AtomGroup
li_atoms = u.atoms.select_atoms("type 22")

# define solvent AtomGroups
PF6 = u.atoms.select_atoms("byres type 21")
BN = u.atoms.select_atoms("byres type 5")
FEC = u.atoms.select_atoms("byres type 19")

solution = Solution(li_atoms, {'PF6': PF6, 'BN': BN, 'FEC': FEC}, radii={'PF6': 2.6})

solution.run()

/Users/orioncohen/projects/development/solvation-analysis/solvation_analysis/solvation_analysis/solution.py:247: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pairs[:, 1] = solvent.ix[[pairs[:, 1]]]


We will be covering the `Residence` and `Networking` analysis objects. These are not instantiated by default with the solution object because they can be very expensive and we wanted to keep `Solution` fast.

That said, `Residence` and `Networking` are very easy to instantiate and add to your analysis when you have a `Solution`! This tutorial covers that process and outlines the key features that these objects offer.

We'll start with `Residence`. We can use the very convenient `from_solution` object to instantiate this.

In [2]:
from solvation_analysis.analysis_library import Residence

residence = Residence.from_solution(solution)

/Users/orioncohen/miniconda3/envs/solvation_analysis/lib/python3.8/site-packages/pandas/core/apply.py:131: FutureWarning: the 'unbiased'' keyword is deprecated, use 'adjusted' instead
  return func(x, *args, **kwargs)
/Users/orioncohen/projects/development/solvation-analysis/solvation_analysis/solvation_analysis/analysis_library.py:589: UserWarning: the autocovariance for BN does not converge to zero so a residence time cannot be calculated. A longer simulation is required to get a valid estimate of the residence time.
  warnings.warn(f'the autocovariance for {res_name} does not converge to zero '
/Users/orioncohen/projects/development/solvation-analysis/solvation_analysis/solvation_analysis/analysis_library.py:589: UserWarning: the autocovariance for PF6 does not converge to zero so a residence time cannot be calculated. A longer simulation is required to get a valid estimate of the residence time.
  warnings.warn(f'the autocovariance for {res_name} does not converge to zero '


Now that we instantiated residence, we can view the residence times of all the solvents!

Note that the residence times here are NOT meaningful, because we are using a snapshot of the simulation that is far too short.

In [3]:
residence.residence_times

{'BN': nan, 'FEC': 6, 'PF6': 9}

We have two methods for binding the `Residence` object to our `Solution` object. We can either "monkey patch" the residence object directly into `Solution`.

In [4]:
solution.residence = residence
solution.residence

Or we can instantiate our Solution with the Residence object included, this is not done by default because Residence can be quite expensive to create.

This approach to instatiating analyis classes with `from_solution` or specifying them in the `analysis_classes` keyword applies to all the analysis classes in `solvation-analyis`,.

In [5]:
solution = Solution(
    li_atoms,
    {'PF6': PF6, 'BN': BN, 'FEC': FEC},
    radii={'PF6': 2.6},
    analysis_classes=["pairing", "coordination", "speciation", "residence"],
)
solution.run()

Networking follows an identical setup pattern but for the networking object, we need to specify which solvents we want to participate in the network. For example, we might be interest in a network of cations and anions in a battery electrolyte. It wouldn't make sense to include the solvents in the network, so we can specify to only look at networking with the anion, which is what we do here.

In [6]:
from solvation_analysis.analysis_library import Networking

networking = Networking.from_solution(solution, 'PF6')

Now that we have the network, lets examine the core data structure of the `Networking` object.

In [7]:
networking.network_df

res_name res_ix
frame network                
0     0            PF6    603
      0         solute    683
      0         solute    690
      1            PF6    616
      1         solute    670
...                ...    ...
9     1         solute    693
      2            PF6    623
      2         solute    664
      3            PF6    648
      3         solute    668

[112 rows x 2 columns]

The `network_df` includes all of the solutes coordinated with at least one solvent, if these coordinated solvents form a network, they are grouped together into a single item in the `network` column of the dataframe.

Here, we can see that in frame 0, our 0th network has two solutes and one PF6 solvent and our 1st network has one solute and one solvent.

The second column lists the residue indexes (from the MDAnalysis Universe) of each listed species.

`Networking` also calculates several convenient statistics, such as the clusters size distribution:

In [8]:
networking.network_sizes

res_name,2,3
frame,,
0,4,2
1,4,1
2,5,1
3,6,0
4,6,0
5,3,0
6,5,0
7,7,0
8,4,1


and the "solute status", or whether the solute is uncoordinated, coordinated with a single solvent, or in a network.

In [9]:
networking.solute_status

{'alone': 0.8918367346938776,
 'paired': 0.09591836734693877,
 'in_network': 0.012244897959183671}

As before, we can instantiate a `Networking` object in the solution directly, but this time, we'll need to specify the solvent(s) of interest in the `networking_solvents` kwarg.

In [10]:
solution = Solution(
    li_atoms,
    {'PF6': PF6, 'BN': BN, 'FEC': FEC},
    radii={'PF6': 2.6},
    analysis_classes=["pairing", "coordination", "speciation", "networking"],
    networking_solvents='PF6'
)
solution.run()

/Users/orioncohen/projects/development/solvation-analysis/solvation_analysis/solvation_analysis/solution.py:247: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pairs[:, 1] = solvent.ix[[pairs[:, 1]]]


Finally, we can effectively integrate the networking object into a visualization workflow by returning the residue indexes of a specific cluster and using them to select the atoms in the cluster.

For this, we'll need to quickly spin up a different solution to use as an example.

In [11]:
from solvation_analysis.tests import datafiles

# instantiate Universe
u = mda.Universe(datafiles.ea_fec_pdb, datafiles.ea_fec_dcd)

# define solute AtomGroup
li_atoms = u.atoms.select_atoms("element Li")

# define solvent AtomGroups
EA = u.residues[0:235].atoms                    # ethyl acetate
FEC = u.residues[235:600].atoms                 # fluorinated ethylene carbonate
PF6 = u.atoms.select_atoms("byres element P")   # hexafluorophosphate

# instantiate solution
solution2 = Solution(li_atoms, 
                    {'EA': EA, 'FEC': FEC, 'PF6': PF6}, 
                    radii={'PF6': 2.6, 'FEC': 2.7})

solution2.run()

In [12]:
networking = Networking.from_solution(solution2, ['FEC', 'PF6'])

In [13]:
networking = Networking.from_solution(solution2, ['PF6'])

In [14]:
networking.network_df[100:150]

res_name res_ix
frame network                
8     0            PF6    665
      0         solute    606
      1            PF6    674
      1         solute    602
      2            PF6    686
      2         solute    630
      3            PF6    691
      3         solute    617
      4            PF6    697
      4         solute    618
      5            PF6    716
      5         solute    601
9     0            PF6    665
      0         solute    606
      1            PF6    674
      1         solute    602
      2            PF6    686
      2         solute    630
      3            PF6    691
      3         solute    617
      4            PF6    697
      4         solute    618
      5            PF6    716
      5         solute    601

In [15]:
# import nglview
import nglview as nv

def visualize(atom_group):
    mda_view = nv.show_mdanalysis(atom_group)
    return mda_view.display()

res_ix = networking.get_network_res_ix(6, 0)
network = solution2.u.residues[res_ix.astype(int)].atoms

visualize(network)

NGLWidget(max_frame=9)